In [ ]:
# install the libraries for crwaling
!pip install requests   # HTTP 요청을 보내는 라이브러리
!pip install beautifulsoup4 # HTML을 파싱하는 라이브러리
!pip install selenium   # 동적 크롤링을 위한 라이브러리
!pip install webdriver_manager  # 크롬 드라이버를 설치하는 라이브러리
!pip install pandas  # 데이터를 다루는 라이브러리

In [1]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options

In [2]:
user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
headers = {
    "User-Agent": user_agent
}

## BS4를 활용한 기사제목 수집

In [4]:
# 키워드를 입력하여 URL 생성
query = input("검색어를 입력하세요: ")
start_date = re.sub(r'[^0-9]', '', input("시작 날짜를 yyyy.mm.dd 형식으로 입력하세요: ")) # 시작 날짜(정규식을 이용하여 숫자만 추출)
end_date = re.sub(r'[^0-9]', '', input("종료 날짜를 yyyy.mm.dd 형식으로 입력하세요: "))    # 종료 날짜(정규식을 이용하여 숫자만 추출)

num_articles_per_page = 15  # 페이지당 기사 수
num_pages = int(input("크롤링할 페이지 수를 입력하세요: "))  # 크롤링할 페이지 수

# 데이터를 저장할 리스트 초기화
title_list = []

for page in tqdm(range(num_pages), desc='Pages...'):
    start_index = page * num_articles_per_page + 1  # 페이지의 시작 인덱스 계산

    # URL 생성
    # url = f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={query}&start={start_index}&pd=3&ds={start_date}&de={end_date}'
    url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query=' + \
        '%s' % query + "&start=" + str(start_index) + "&pd=3&ds=" + \
        '%s' % start_date + "&de=" + '%s' % end_date

    print(f'URL: {url}')

    # HTML 문서 가져오기
    response = requests.get(url, headers=headers);  print(f'status code: {response.status_code}')
    html = response.text

    # BeautifulSoup을 이용하여 HTML 파싱
    soup = bs(html, 'html.parser'); 

    # 기사 제목 추출
    titles = soup.find_all('a', class_="news_tit")

    # 추출한 데이터를 리스트에 저장
    for title in tqdm(titles, desc="Links..."):
        print(title.text)
        title_list.append(title.text)
        
    print(f'{page}번째 페이지')
    print(f'기사 갯수: {len(title_list)}')
    print(f'{"-"*200}')
    time.sleep(rd.uniform(2.5, 3.5))
        
print(f'총 기사 갯수: {len(title_list)}')
print(title_list)

# 데이터프레임 생성
df = pd.DataFrame(title_list, columns=["뉴스 제목"])

# CSV 파일로 저장
df.to_csv(f'../data/{query}_{start_date}~{end_date}_{num_pages}page.csv', index=False)

Pages...:   0%|          | 0/334 [00:00<?, ?it/s]

URL: https://search.naver.com/search.naver?where=news&sm=tab_jum&query=오염수&start=1&pd=3&ds=20200101&de=20211231
status code: 200


Links...: 100%|██████████| 15/15 [00:00<?, ?it/s]

 [신년사]문성혁 해수부 장관 "새해 첨단 항만인프라 선제적으로 구축" 
 올해 후쿠시마 방사능 오염수 30% 감소…해양 방류 늦춰질까 
 원전 지붕 수리했더니 후쿠시마 오염수 감소…방출 시기 늦출까 
 후쿠시마 오염수 불안에… 日, 주변국 소비자 인식조사 한다 
 중국, 일본 원전 오염수 방류 결정에 "조속히 철회해야" 
 [사이언스 HOT5] 日 후쿠시마 오염수 방류 준비…12월 다섯째주 과학 이슈 
 후쿠시마 오염수 ‘최대 난관’ 어민 달래려 3천억원 기금마련 
 日 방사능 오염수, 움직이는 배로 실시간 감시한다 
 배 띄워 '방사능 오염수' 감시‥"실시간 분석" 
 후쿠시마 오염수 불안에…수산물 이력제 대폭 개편 
 외교부, '후쿠시마 오염수 방류신청' 관련 일본 공사 초치 
 통영거제환경련, 올해 주요 환경 이슈 발표 
 [경제직필]이렇게 해선 일본 오염수 방출 못 막아 
 정부, '日 오염수 방류' 준비에 "심각한 유감, 정당성 가져야"(종합) 
 日도쿄전력, 오늘 후쿠시마 오염수 해양방류 심사신청 
0번째 페이지
기사 갯수: 15
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



Pages...:   0%|          | 1/334 [00:03<17:42,  3.19s/it]

...

URL: https://search.naver.com/search.naver?where=news&sm=tab_jum&query=오염수&start=3031&pd=3&ds=20200101&de=20211231
status code: 200


Links...: 100%|██████████| 15/15 [00:00<00:00, 14997.51it/s]

 울진공무원노조, 日 원전오염수 방류 즉각 철회 촉구 
 "후쿠시마 원전 오염수 방류 결정 철회하라" 
 '방사능 오염수 방류 막아내자'...제주 어업인 소송 청구 
 오염수 방출, 자민당서도 반발…변명만 늘어놓은 日정부 
 안산시민사회연대, 일본 후쿠시마 오염수 해양방류 규탄 
 선원노련 등 노동계도 '日 방사능 오염수 방류' 규탄 
 "방사성 오염수, 해양생태계 위험에 빠뜨릴 것" 
 안산시, 31일 '일본 원전 오염수 방류 규탄' 해상 시위(종합) 
 포항시, 전국 지자체 최초 수산물 품질 검사기관에 지정 
 "일본 방사능 오염수 방류 막아내야... 일본 두둔 미국도 규탄 대상" 
 정종순 장흥군수, 일본 방사능 오염수 방류 규탄 챌린지 동참 
 후쿠시마 원전 오염수 방류, 우리의 과제는? 
 일본의 방사능 오염수 방류, 충북도 반대다 
 일본은 방사성 물질 오염수 배출을 철회하라 
 권덕철 복지 장관, 국제 사회에 '백신 생산 확대'·'日 오염수 방출 정보 공개' 촉구 
202번째 페이지
기사 갯수: 3045
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------



Pages...:  60%|██████    | 202/334 [11:55<07:47,  3.54s/it]


KeyboardInterrupt: 